In [653]:
import csv
import re
import numpy as np
import pickle

READ CSV

In [654]:
articles_id = []
titles = []
contents = []

with open("aritcles.csv", "r") as a:
    aritcles = csv.reader(a)

    for row in aritcles:
        if row[0] == "id":
            continue
        
        articles_id.append(row[0])
        titles.append(row[1])

        # print(len(row))
        content = ""
        for i in range(2, len(row)):
            content += row[i] + " "
        
        contents.append(content)

    print(contents)

['Artificial intelligence is transforming industries globally. Machine learning and deep learning are key componentsâ€¦ ', 'Robotics is advancing rapidly  integrating with AI for autonomous systems. Human-robot interaction is a growing fieldâ€¦ ', ' Data Engineering leverages tools and computation to move massive amounts of data. Big data analytics is crucialâ€¦ ']


CLEAN

In [655]:
cleaned_contents = []

for text in contents:
    text = text.lower()
    text = re.sub("[^a-z ]", "", text)
    cleaned_contents.append(text.strip())

print(cleaned_contents)

['artificial intelligence is transforming industries globally machine learning and deep learning are key components', 'robotics is advancing rapidly  integrating with ai for autonomous systems humanrobot interaction is a growing field', 'data engineering leverages tools and computation to move massive amounts of data big data analytics is crucial']


TOKENIZE

In [656]:
tokenized_articles = []
for words in cleaned_contents:
    tokenized_articles.append(words.split())

print(tokenized_articles)

[['artificial', 'intelligence', 'is', 'transforming', 'industries', 'globally', 'machine', 'learning', 'and', 'deep', 'learning', 'are', 'key', 'components'], ['robotics', 'is', 'advancing', 'rapidly', 'integrating', 'with', 'ai', 'for', 'autonomous', 'systems', 'humanrobot', 'interaction', 'is', 'a', 'growing', 'field'], ['data', 'engineering', 'leverages', 'tools', 'and', 'computation', 'to', 'move', 'massive', 'amounts', 'of', 'data', 'big', 'data', 'analytics', 'is', 'crucial']]


BAG OF WORDS

In [657]:
bag_of_words = []

for article in tokenized_articles:
    for word in article:
        if word not in bag_of_words:
            bag_of_words.append(word)

print(bag_of_words)

['artificial', 'intelligence', 'is', 'transforming', 'industries', 'globally', 'machine', 'learning', 'and', 'deep', 'are', 'key', 'components', 'robotics', 'advancing', 'rapidly', 'integrating', 'with', 'ai', 'for', 'autonomous', 'systems', 'humanrobot', 'interaction', 'a', 'growing', 'field', 'data', 'engineering', 'leverages', 'tools', 'computation', 'to', 'move', 'massive', 'amounts', 'of', 'big', 'analytics', 'crucial']


TEXT to VECTOR

In [658]:

def text_to_vector(article_words, the_bag_of_words):

    vector = []
    for word in the_bag_of_words:
        if word in article_words:
            vector.append(1)
        else:
            vector.append(0)
    
    return vector

vectors = []
for i in range(len(tokenized_articles)):
    vectors.append( text_to_vector(tokenized_articles[i], bag_of_words) )

print(vectors)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


COSINE SIMILARITY

In [659]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)

    return dot_product / (norm1 * norm2)

v1 = text_to_vector(contents[0], bag_of_words)
v2 = text_to_vector(contents[1], bag_of_words)

print( cosine_similarity(v1, v2) )

0.2307692307692308


SIMILARITY MATRIX

In [660]:
similarity_matrix = []
n = len(vectors)
for i in range(n):

    row = []
    for j in range(n):
        row.append( ( cosine_similarity(vectors[i], vectors[j]) ) )
    
    similarity_matrix.append(row)

similarity_matrix = np.array(similarity_matrix)
print(similarity_matrix)

[[1.         0.07161149 0.14322297]
 [0.07161149 1.         0.06666667]
 [0.14322297 0.06666667 1.        ]]


SAVE to PKL

In [661]:
# WRITE 
with open("Similarity_Matrix.pkl", "wb") as file:
    pickle.dump(similarity_matrix, file)

# READ
with open("Similarity_Matrix.pkl", "rb") as file:
    print(pickle.load(file))

[[1.         0.07161149 0.14322297]
 [0.07161149 1.         0.06666667]
 [0.14322297 0.06666667 1.        ]]


MOST SIMILAR ARTICLES

In [662]:

def most_similar(article_id = 1):

    index = articles_id.index( str(article_id) )
    Similarities = similarity_matrix[index]
    Similarities_sorted = sorted(Similarities, reverse=True)

    best_article_score = Similarities_sorted[1]
    best_article_index = np.where(similarity_matrix[index] == best_article_score)
    best_article_id = articles_id[ best_article_index[0][0] ]
    best_article = titles[best_article_index[0][0]]

    print(f"The Most Similar Article to '{titles[index]}', ID: {article_id}  is \n'{best_article}',  ID: {best_article_id}")


most_similar(3)

The Most Similar Article to 'Data Engineering', ID: 3  is 
'The Rise of AI',  ID: 1
